In [14]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
%matplotlib inline

cnx = sqlite3.connect('database.sqlite')
# Create copies of the tables
df_player_attr_orig = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
df_player_orig = pd.read_sql_query("SELECT * FROM Player", cnx)
df_country_orig = pd.read_sql_query("SELECT * FROM country", cnx)
df_league_orig = pd.read_sql_query("SELECT * FROM league", cnx)
df_match_orig = pd.read_sql_query("SELECT * FROM match", cnx)
df_team_orig = pd.read_sql_query("SELECT * FROM team", cnx)
df_team_attr_orig = pd.read_sql_query("SELECT * FROM team_Attributes", cnx)

# Create copies to manipulate
df_match = df_match_orig.copy()
df_player_attr = df_player_attr_orig.copy()

# Now drop rows with NAN values
match_rows = df_match.shape[0]
df_match.dropna(inplace=True)
print('df_match started with {} row, end with {} row, difference of {} rows'.format(match_rows, 
                                                                                    df_match.shape[0], 
                                                                                    match_rows-df_match.shape[0]))
player_attr_rows = df_player_attr.shape[0]
df_player_attr.dropna(inplace=True)
print('df_player_attr started with {} row, end with {} row, difference of {} rows'.format(player_attr_rows, 
                                                                                    df_player_attr.shape[0], 
                                                                                    player_attr_rows-df_player_attr.shape[0]))



df_match started with 25979 row, end with 1762 row, difference of 24217 rows
df_player_attr started with 183978 row, end with 180354 row, difference of 3624 rows


Now that null values removed, build functions to massage data to get:
- Win/tie/loss for home and way team
- Convert date into python date and be available for joins and use to create new table with group by
- Get ratings for all home and away players
- Get max and min rating for home and away teams

In [16]:
# First get win/tie/loss
# REFACTOR ALL CODE TO USE ADDITIONAL PARAMETERS

def match_result(row, team):
    if team == 'home':
        if row['home_team_goal'] > row['away_team_goal']:
            return 'w'
        elif row['home_team_goal'] == row['away_team_goal']:
            return 't'
        else:
            return 'l'
    else:
        if row['away_team_goal'] > row['home_team_goal']:
            return 'w'
        elif row['away_team_goal'] == row['home_team_goal']:
            return 't'
        else:
            return 'l'

df_match['home_match_result'] = df_match.apply(match_result, args=('home',), axis=1)
df_match['away_match_result'] = df_match.apply(match_result, args=('away',), axis=1)


In [17]:
# Second get date time

def convert_dt(row):
    return datetime.datetime.strptime(row, '%Y-%m-%d %H:%M:%S')

# Convert date of player rating using function and then create new dataframe of player's rating
df_player_attr['dt'] = df_player_attr.date.apply(convert_dt)
df_player_attr['month'] = df_player_attr.dt.apply(lambda x: x.month)
df_player_attr['year'] = df_player_attr.dt.apply(lambda x: x.year)

# Create new table
df_player_year_rating = df_player_attr[['player_api_id', 'overall_rating', 'year']].groupby(['year', 'player_api_id']).mean().reset_index()

# Convert date of match to be able to join to the player year rating table
df_match['dt'] = df_match.date.apply(convert_dt)
df_match['month'] = df_match.dt.apply(lambda x: x.month)
df_match['year'] = df_match.dt.apply(lambda x: x.year)

In [18]:
# Third, define a function to get the player rating
def get_player_rating(df, home_away, pos_num):
    cols = df.columns.tolist()
    join_col = '{}_player_{}'.format(home_away, pos_num)
    df = df.merge(df_player_year_rating, how='inner', left_on=[join_col, 'year'], right_on=['player_api_id', 'year'])
    df['{}_rating'.format(join_col)] = df['overall_rating']
    df.drop(['player_api_id', 'overall_rating'], axis=1, inplace=True)
    return df

for loc in ['home', 'away']:
    for x in range(1, 12):
        df_match = get_player_rating(df_match, loc, x)

In [36]:
# Fourthbuild a function to go over the columns and add a home_player_max_rating, home_player_min_rating, 
# an away_player_max_rating and an away_player_min_rating
# REFACTOR TO TAKE EXTRA ARGS

def get_max_rating(row, team):
    max_rating = 0
    for x in range(1, 12):
        rating = row['{}_player_{}_rating'.format(team, x)]
        if rating > max_rating:
            max_rating = rating
        else:
            pass
    return max_rating

def get_min_rating(row, team):
    min_rating = 100
    for x in range(1, 12):
        rating = row['{}_player_{}_rating'.format(team, x)]
        if rating < min_rating:
            min_rating = rating
        else:
            pass
    return min_rating

def get_avg_rating(row, team, size, min_max):
    rating_list = []
    for x in range(1, 12):
        rating_list.append(row['{}_player_{}_rating'.format(team, x)])
    
    if min_max == 'max':
        rating_list.sort(reverse=True)
    else:
        rating_list.sort(reverse=False)
    
    return mean(rating_list[:size])

def get_home_std_rating(row):
    rating_list = []
    for x in range(1, 12):
        rating_list.append(row['home_player_{}_rating'.format(x)])
        
    return np.std(rating_list)

def get_away_std_rating(row):
    rating_list = []
    for x in range(1, 12):
        rating_list.append(row['away_player_{}_rating'.format(x)])
        
    return np.std(rating_list)


df_match['home_player_max_rating'] = df_match.apply(get_max_rating, args=('home',), axis=1)
df_match['home_player_min_rating'] = df_match.apply(get_min_rating, args=('home',), axis=1)
df_match['away_player_max_rating'] = df_match.apply(get_max_rating, args=('away',), axis=1)
df_match['away_player_min_rating'] = df_match.apply(get_min_rating, args=('away',), axis=1)
df_match['home_player_avg_rating'] = df_match.apply(get_avg_rating, args=('home', 12, 'max'), axis=1)
df_match['away_player_avg_rating'] = df_match.apply(get_avg_rating, args=('away', 12, 'max'), axis=1)
df_match['home_player_avg_top3_rating'] = df_match.apply(get_avg_rating, args=('home', 4, 'max'), axis=1)
df_match['away_player_avg_top3_rating'] = df_match.apply(get_avg_rating, args=('away', 4, 'max'), axis=1)
df_match['home_player_avg_bot3_rating'] = df_match.apply(get_avg_rating, args=('home', 4, 'min'), axis=1)
df_match['away_player_avg_bot3_rating'] = df_match.apply(get_avg_rating, args=('away', 4, 'min'), axis=1)


In [37]:
df_match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,home_player_max_rating,home_player_min_rating,away_player_max_rating,away_player_min_rating,home_player_avg_rating,away_player_avg_rating,home_player_avg_top3_rating,away_player_avg_top3_rating,home_player_avg_bot3_rating,away_player_avg_bot3_rating
0,3249,1729,1729,2012/2013,1,2012-08-18 00:00:00,1228229,9825,8472,0,...,84.0,64.5,81.5,71.0,79.090909,74.818182,82.625,78.000,74.000,72.000
1,3601,1729,1729,2012/2013,7,2012-10-06 00:00:00,1229164,8456,8472,3,...,88.0,78.0,81.5,71.0,82.454545,75.681818,85.750,79.125,79.375,72.500
2,3627,1729,1729,2012/2013,9,2012-10-27 00:00:00,1229194,10194,8472,0,...,78.5,70.0,80.0,71.0,75.318182,74.818182,77.375,78.000,72.875,72.000
3,3474,1729,1729,2012/2013,3,2012-09-01 00:00:00,1228306,10003,8472,2,...,81.0,70.5,81.5,71.0,74.681818,75.636364,76.875,79.125,72.375,72.375
4,3331,1729,1729,2012/2013,17,2012-12-15 00:00:00,1229275,10260,8472,3,...,89.5,77.5,81.5,71.0,82.136364,75.727273,86.125,79.125,78.125,72.625


Now all the information is available. Look at:
- Who had highest rating and compare to who won
- Who had lowest rating and compare to who lost
- Break the above two into home/away and see if the data changes

In [7]:
def highest_rating_won(row):
    away_r = row['away_player_max_rating']
    home_r = row['home_player_max_rating']
    match_home = row['home_match_result']
    
    if home_r > away_r and match_home == 'w':
        return 1
    elif home_r < away_r and match_home == 'l':
        return 1
    else:
        return 0
    
df_match['highest_rating_won'] = df_match.apply(highest_rating_won, axis=1)
df_match.groupby(['highest_rating_won']).count()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_player_10_rating,away_player_11_rating,home_player_max_rating,home_player_min_rating,away_player_max_rating,away_player_min_rating,home_player_avg_rating,away_player_avg_rating,home_player_std_rating,away_player_std_rating
highest_rating_won,,,,,,,,,,,,,,,,,,,,,
0,883,883,883,883,883,883,883,883,883,883,...,883,883,883,883,883,883,883,883,883,883
1,827,827,827,827,827,827,827,827,827,827,...,827,827,827,827,827,827,827,827,827,827


So the highest rating won 827 out of (827 + 883)

Now lowest rating and lost

In [8]:
def lowest_rating_lost(row):
    away_r = row['away_player_min_rating']
    home_r = row['home_player_min_rating']
    match_home = row['home_match_result']
    
    if home_r < away_r and match_home == 'l':
        return 1
    elif home_r > away_r and match_home == 'w':
        return 1
    else:
        return 0
    
df_match['lowest_rating_lost'] = df_match.apply(lowest_rating_lost, axis=1)
df_match.groupby(['lowest_rating_lost']).count()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_player_11_rating,home_player_max_rating,home_player_min_rating,away_player_max_rating,away_player_min_rating,home_player_avg_rating,away_player_avg_rating,home_player_std_rating,away_player_std_rating,highest_rating_won
lowest_rating_lost,,,,,,,,,,,,,,,,,,,,,
0,935,935,935,935,935,935,935,935,935,935,...,935,935,935,935,935,935,935,935,935,935
1,775,775,775,775,775,775,775,775,775,775,...,775,775,775,775,775,775,775,775,775,775


So the lowest rating lost 775 out of 775+935

Now look at average rating

In [9]:
def highest_avg_rating_won(row):
    away_r = row['away_player_avg_rating']
    home_r = row['home_player_avg_rating']
    match_home = row['home_match_result']
    
    if home_r > away_r and match_home == 'w':
        return 1
    elif home_r < away_r and match_home == 'l':
        return 1
    else:
        return 0
    
df_match['highest_avg_rating_won'] = df_match.apply(highest_avg_rating_won, axis=1)
df_match.groupby(['highest_avg_rating_won']).count()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,home_player_max_rating,home_player_min_rating,away_player_max_rating,away_player_min_rating,home_player_avg_rating,away_player_avg_rating,home_player_std_rating,away_player_std_rating,highest_rating_won,lowest_rating_lost
highest_avg_rating_won,,,,,,,,,,,,,,,,,,,,,
0,841,841,841,841,841,841,841,841,841,841,...,841,841,841,841,841,841,841,841,841,841
1,869,869,869,869,869,869,869,869,869,869,...,869,869,869,869,869,869,869,869,869,869


In [10]:
def lowest_std_rating_won(row):
    away_r = row['away_player_std_rating']
    home_r = row['home_player_std_rating']
    match_home = row['home_match_result']
    
    if home_r < away_r and match_home == 'w':
        return 1
    elif home_r > away_r and match_home == 'l':
        return 1
    else:
        return 0
    
df_match['lowest_std_rating_won'] = df_match.apply(lowest_std_rating_won, axis=1)
df_match.groupby(['lowest_std_rating_won']).count()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,home_player_min_rating,away_player_max_rating,away_player_min_rating,home_player_avg_rating,away_player_avg_rating,home_player_std_rating,away_player_std_rating,highest_rating_won,lowest_rating_lost,highest_avg_rating_won
lowest_std_rating_won,,,,,,,,,,,,,,,,,,,,,
0,1126,1126,1126,1126,1126,1126,1126,1126,1126,1126,...,1126,1126,1126,1126,1126,1126,1126,1126,1126,1126
1,584,584,584,584,584,584,584,584,584,584,...,584,584,584,584,584,584,584,584,584,584


In [13]:
related = df_match['home_player_avg_rating'].corr(df_match['home_match_result'])
print("%s: %f" % (f,related))


TypeError: unsupported operand type(s) for /: 'str' and 'int'

Ideas for questions:
- How do the individual player attributes impact team attributes?
- How often does the team with the highest rate player win?
- How does the average player rating of the team affect the team winning?
- Does the team with the highest rated player win more than the team with a better overall rating?
- What correlates better with match results: does the team with the highest player win more, the team with the lowest player lose more, or the team with the highest average player rating win more? (Break this out too into does it matter if the team is home or away? And could look at by positional groups, does team with best forward, best midfielder, best defender vs worst


Question to answer: which of the following correlates to match result best: the team with the highest player, highest average player, smallest std in player rating, or lowest player 

How do player ratings correlate to wins and losses? In particular, does having the best player correlate better to winning? The least worst player? The highest average rating?

Like most major sports, soccer is a team sport. But with team sports, the question is how much does the quality of an individual player or individual players influence the outcome  of the match? Is it just about having the best player? The best 1-3 players (on average)? Or is the opposite? Having the least worst player? Or least worst 1-3 (on average)?